<a href="https://colab.research.google.com/github/angiequima/Econometria_QLab_PC/blob/main/PC1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Ejercicio 1**


In [3]:
# Instalamos e importamos librerías
%pip install linearmodels

In [4]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Cargamos la base de datos
df = pd.read_stata('C:/Users/LESLY/Downloads/eitc.dta')

In [ ]:
# importamos

import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# Cargamos la base de datos
df = pd.read_stata('datacbp.dta')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38606 entries, 0 to 38605
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fipstate  38606 non-null  int8   
 1   naics     38606 non-null  object 
 2   emp       38606 non-null  int32  
 3   qp1       38606 non-null  int32  
 4   ap        38606 non-null  int32  
 5   est       38606 non-null  int32  
 6   year      38606 non-null  float32
dtypes: float32(1), int32(4), int8(1), object(1)
memory usage: 1.1+ MB


In [11]:
df

,fipstate,naics,emp,qp1,ap,est,year
0,1,113///,8207,44757,195720,1082,1998.0
1,1,114///,93,363,1532,30,1998.0
2,1,115///,1614,10181,45519,194,1998.0
3,1,211///,804,9477,38146,38,1998.0
4,1,212///,7074,79136,318295,159,1998.0
...,...,...,...,...,...,...,...
38601,56,721///,7759,30345,202369,499,2006.0
38602,56,722///,17741,46549,211741,1217,2006.0
38603,56,811///,3274,23666,110002,721,2006.0
38604,56,812///,1721,7451,31694,366,2006.0


#### Descripción de las variables
* fipstate: código estatal
* naics: código de industria
* emp: total de empleados a mediados de marzo
* qp1: nómina total del primer trimestre
* ap: nómina anual total
* est: número total de establecimientos

In [12]:
df['year']

,year
0,1998.0
1,1998.0
2,1998.0
3,1998.0
4,1998.0
...,...
38601,2006.0
38602,2006.0
38603,2006.0
38604,2006.0


In [24]:
# Restringimos la muestra y concatenamos

processed_yearly_dfs = []

for year_val in range(1998, 2007):
    # Filtrar datos para el año actual
    df_year = df[df['year'] == year_val].copy()

    # Limpiar la columna "naics" para conservar solo los primeros tres caracteres numéricos
    df_year['naics'] = df_year['naics'].astype(str).str[:3]
    df_year = df_year[df_year['naics'].str.isnumeric()].copy()

    # Seleccionar columnas específicas
    selected_columns = ['fipstate', 'year', 'naics', 'emp', 'qp1', 'ap', 'est']
    df_year_processed = df_year[selected_columns]

    processed_yearly_dfs.append(df_year_processed)

# Concatenar todos los marcos de datos anuales procesados ​​en un único marco de datos
processed_df = pd.concat(processed_yearly_dfs, ignore_index=True)

processed_df.info()
display(processed_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38606 entries, 0 to 38605
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fipstate  38606 non-null  int8   
 1   year      38606 non-null  float32
 2   naics     38606 non-null  object 
 3   emp       38606 non-null  int32  
 4   qp1       38606 non-null  int32  
 5   ap        38606 non-null  int32  
 6   est       38606 non-null  int32  
dtypes: float32(1), int32(4), int8(1), object(1)
memory usage: 1.1+ MB


,fipstate,year,naics,emp,qp1,ap,est
0,1,1998.0,113,8207,44757,195720,1082
1,1,1998.0,114,93,363,1532,30
2,1,1998.0,115,1614,10181,45519,194
3,1,1998.0,211,804,9477,38146,38
4,1,1998.0,212,7074,79136,318295,159


###1. El nivel de observación de `processed_df` es a nivel de **fipstate-year-naics** (de 3 dígitos).

### 2. Variable ficticia `post_china`

In [30]:
processed_df['post_china'] = (processed_df['year'] >= 2001).astype(int)
processed_df

,fipstate,year,naics,emp,qp1,ap,est,post_china
0,1,1998.0,113,8207,44757,195720,1082,0
1,1,1998.0,114,93,363,1532,30,0
2,1,1998.0,115,1614,10181,45519,194,0
3,1,1998.0,211,804,9477,38146,38,0
4,1,1998.0,212,7074,79136,318295,159,0
...,...,...,...,...,...,...,...,...
38601,56,2006.0,721,7759,30345,202369,499,1
38602,56,2006.0,722,17741,46549,211741,1217,1
38603,56,2006.0,811,3274,23666,110002,721,1
38604,56,2006.0,812,1721,7451,31694,366,1


###3. Variable ficticia `manuf`

In [32]:
processed_df['manuf'] = (processed_df['naics'].astype(str).str.startswith('3')).astype(int)
processed_df

,fipstate,year,naics,emp,qp1,ap,est,post_china,manuf
0,1,1998.0,113,8207,44757,195720,1082,0,0
1,1,1998.0,114,93,363,1532,30,0,0
2,1,1998.0,115,1614,10181,45519,194,0,0
3,1,1998.0,211,804,9477,38146,38,0,0
4,1,1998.0,212,7074,79136,318295,159,0,0
...,...,...,...,...,...,...,...,...,...
38601,56,2006.0,721,7759,30345,202369,499,1,0
38602,56,2006.0,722,17741,46549,211741,1217,1,0
38603,56,2006.0,811,3274,23666,110002,721,1,0
38604,56,2006.0,812,1721,7451,31694,366,1,0


###4. ¿Cuál es el efecto de la entrada de China en la OMC sobre el empleo?
**Grupo de Tratamiento**: Industrias manufactureras (`manuf = 1`)

**Grupo de Control**: Industrias no manufactureras (`manuf = 0`)

**Período Post-intervención**: Año >= 2001 (`post_china = 1`)

**Período Pre-intervención**: Año < 2001 (`post_china = 0`)

In [33]:
# Calculamos el empleo promedio para cada una de las cuatro celdas
did_means = processed_df.groupby(['manuf', 'post_china'])['emp'].mean().unstack()

print("Empleo promedio (emp) por grupo y período:")
display(did_means)

Empleo promedio (emp) por grupo y período:


post_china,0,1
manuf,,
0,29539.341747,30579.647311
1,15619.126095,13366.851284


In [40]:
# Calculamos el efecto DiD
# (Empleo promedio post-tratamiento - Empleo promedio pre-tratamiento) - (Empleo promedio post-control - Empleo promedio pre-control)
did_effect = (did_means.loc[1, 1] - did_means.loc[1, 0]) - (did_means.loc[0, 1] - did_means.loc[0, 0])

print(f"El efecto de diferencias en diferencias (DiD) sobre el empleo es: {did_effect:.2f}")

El efecto de diferencias en diferencias (DiD) sobre el empleo es: -3292.58


**Interpretación**: El empleo en las industrias manufactureras disminuyó *más* en el período post-tratamiento (entrada de China en la OMC) en comparación con las industrias no manufactureras. Es decir, la entrada de China en la OMC sobre el empleo en el sector manufacturero tuvo un impacrto negativo.

###5. Regresión de DiD

In [39]:
import statsmodels.formula.api as smf

# Fórmula de la regresión DiD: emp = beta_0 + beta_1*manuf + beta_2*post_china + beta_3*(manuf * post_china)
formula_did = 'emp ~ manuf + post_china + manuf:post_china'

# Ajustar el modelo OLS
model_did = smf.ols(formula=formula_did, data=processed_df).fit()

print("\n--- RESULTADOS DE LA REGRESIÓN DiD ---")
print(model_did.summary())


--- RESULTADOS DE LA REGRESIÓN DiD ---
                            OLS Regression Results                            
Dep. Variable:                    emp   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     176.0
Date:                Tue, 03 Feb 2026   Prob (F-statistic):          2.44e-113
Time:                        23:26:14   Log-Likelihood:            -4.7943e+05
No. Observations:               38606   AIC:                         9.589e+05
Df Residuals:                   38602   BIC:                         9.589e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------


Se puede observar que el coeficiente beta_3 o `manuf:post_china` del ítem 5 coincide con el resultado del ítem 4 que es -3292.5804

#**Ejercicio 2**

## 1: Tabla resumen de todos los datos

In [ ]:
print("--- RESUMEN DE DATOS ---")
# describe() nos da media, desviación estándar, min, max, etc.
summary_table = df.describe()
display(summary_table)

--- RESUMEN DE DATOS ---


,state,year,urate,children,nonwhite,finc,earn,age,ed,work,unearn
count,13746.00000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000
mean,54.52459,1993.347046,6.761734,1.192638,0.600684,15255.319310,10432.475514,35.209661,8.806053,0.513022,4.822844
std,27.13521,1.709800,1.462461,1.382105,0.489776,19444.249684,18200.758138,10.157128,2.635639,0.499849,7.122624
min,11.00000,1991.000000,2.600000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000
25%,31.00000,1992.000000,5.700000,0.000000,0.000000,5123.417782,0.000000,26.000000,7.000000,0.000000,0.000000
50%,56.00000,1993.000000,6.800000,1.000000,1.000000,9636.664360,3332.179931,34.000000,10.000000,1.000000,2.973197
75%,81.00000,1995.000000,7.700000,2.000000,1.000000,18659.178178,14321.223709,44.000000,11.000000,1.000000,6.863863
max,95.00000,1996.000000,11.400000,9.000000,1.000000,575616.820870,537880.611850,54.000000,11.000000,1.000000,134.057520


## 2: Medias muestrales por grupo de hijos

In [ ]:
print("\n--- MEDIAS POR GRUPO DE HIJOS ---")

# Creamos una función para clasificar a las mujeres según los grupos:
# (a) Sin hijos, (b) 1 hijo, (c) 2 o más hijos

def clasificar_hijos(n):
    if n == 0:
        return 'a) Sin hijos'
    elif n == 1:
        return 'b) 1 hijo'
    else:
        return 'c) 2+ hijos'

# Aplicamos la clasificación
df['grupo_hijos'] = df['children'].apply(clasificar_hijos)

# Calculamos el promedio de TODAS las variables agrupando por esta nueva categoría
medias_por_grupo = df.groupby('grupo_hijos').mean(numeric_only=True).T

display(medias_por_grupo)


--- MEDIAS POR GRUPO DE HIJOS ---


grupo_hijos,a) Sin hijos,b) 1 hijo,c) 2+ hijos
state,53.396660,55.590908,55.243855
year,1993.365479,1993.338135,1993.329956
urate,6.663067,6.802060,6.858664
children,0.000000,1.000000,2.801092
nonwhite,0.515944,0.596468,0.708885
finc,18559.863110,13941.569547,11985.295298
earn,13760.255844,9928.278759,6613.546648
age,38.498228,33.758993,32.047469
ed,8.548676,8.992479,9.006721
work,0.574490,0.537606,0.420710


## 3: Construir variables 'anykids' y 'post93'

In [ ]:
# Variable de tratamiento (anykids): 1 si tiene al menos un hijo, 0 si no
df['anykids'] = (df['children'] >= 1).astype(int)

# Variable temporal (post93): 1 si el año es 1994 o posterior, 0 si no
df['post93'] = (df['year'] >= 1994).astype(int)


In [ ]:
# Creamos el término de interacción (el DiD)
df['interaction'] = df['anykids'] * df['post93']

# Verificamos que se hayan creado bien
print(df[['year', 'children', 'anykids', 'post93', 'interaction']].head())

     year  children  anykids  post93  interaction
0  1991.0         0        0       0            0
1  1991.0         1        1       0            0
2  1991.0         2        1       0            0
3  1991.0         0        0       0            0
4  1991.0         3        1       0            0


## 4: Regresión Diferencias en Diferencias

In [ ]:
# TNuestro modelo sería: earn = beta0 + beta1*anykids + beta2*post93 + beta3*interacción
# El coeficiente de interés es 'interaction' (beta3)

model_basic = smf.ols(formula='earn ~ anykids + post93 + interaction', data=df).fit()

print("\n--- RESULTADOS DE LA REGRESIÓN ---")
print(model_basic.summary())


--- RESULTADOS DE LA REGRESIÓN ---
                            OLS Regression Results                            
Dep. Variable:                   earn   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     124.3
Date:                Tue, 03 Feb 2026   Prob (F-statistic):           1.96e-79
Time:                        11:17:13   Log-Likelihood:            -1.5416e+05
No. Observations:               13746   AIC:                         3.083e+05
Df Residuals:                   13742   BIC:                         3.084e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     

- Interpretación: Viendo el coeficiente de la variable "interaction" (con un p-valor 0.007, que es estadisticamente significativo) podemos decir que, tras la expansión del programa EITC, las ganancias (earnings) de las mujeres solteras con hijos aumentaron en promedio $1669 dólares más que las de las mujeres solteras sin hijos.

## 5: Modelo con Efectos Fijos de Estado y Año

Ahora debemos repetir el modelo (iv) pero con FE de estado y año.

In [ ]:
# Fórmula: earn = anykids + interaction + Efectos Fijos
# C(state) crea dummies por estado, C(year) crea dummies por año.
model_fe = smf.ols(formula='earn ~ anykids + interaction + C(state) + C(year)', data=df).fit()

print("--- RESULTADOS 5 (Efectos Fijos) ---")
# Buscamos y mostramos las filas de 'anykids' e 'interaction'
results_as_html = model_fe.summary().tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]
display(result_df.loc[['anykids', 'interaction']])

--- RESULTADOS (Efectos Fijos) ---


,coef,std err,t,P>|t|,[0.025,0.975]
anykids,-6646.0405,420.540,-15.804,0.000,-7470.357,-5821.724
interaction,1612.1312,617.393,2.611,0.009,401.956,2822.307


- El nuevo coeficiente de interaccion es similar al resultado de la Pregunta 4 (que era casi 1669). Esto sugiere que el efecto estimado es robusto; es decir, no cambia mucho incluso cuando controlamos por las diferencias fijas entre estados y los choques anuales comunes.

## 6: Modelo con Controles Adicionales

In [ ]:
# Agregamos: urate (desempleo), nonwhite, age, ed (educación), unearn (ingreso no laboral)

model_controls = smf.ols(
    formula='earn ~ anykids + interaction + urate + nonwhite + age + ed + unearn + C(state) + C(year)',
    data=df
).fit()

print("\n--- RESULTADOS 6 (Con Controles) ---")
results_as_html_cont = model_controls.summary().tables[1].as_html()
result_df_cont = pd.read_html(results_as_html_cont, header=0, index_col=0)[0]
display(result_df_cont.loc[['anykids', 'interaction', 'urate', 'nonwhite', 'age', 'ed', 'unearn']])


--- RESULTADOS 6 (Con Controles) ---


,coef,std err,t,P>|t|,[0.025,0.975]
anykids,-6224.6707,431.618,-14.422,0.000,-7070.702,-5378.640
interaction,1661.8117,616.237,2.697,0.007,453.903,2869.720
urate,-44.8069,276.405,-0.162,0.871,-586.599,496.985
nonwhite,-2575.0164,361.745,-7.118,0.000,-3284.087,-1865.946
age,33.5963,15.934,2.108,0.035,2.363,64.830
ed,119.9487,60.318,1.989,0.047,1.717,238.180
unearn,-42.7092,21.670,-1.971,0.049,-85.186,-0.233


- Nuevamente, el resultado del coeficiente es muy estable (similar a 1668 y 1612). Esto refuerza la conclusión de que el programa EITC tuvo un efecto positivo y significativo en las ganancias, independientemente de la edad, educación o ubicación de las mujeres.

## 7: Efectos Variados (1 hijo vs 2+ hijos)

In [ ]:
# 1. Creamos dummies para los grupos específicos de hijos
df['one_kid'] = (df['children'] == 1).astype(int)
df['two_plus_kids'] = (df['children'] >= 2).astype(int)

# 2. Creamos las interacciones con post93
df['interact_1'] = df['one_kid'] * df['post93']
df['interact_2'] = df['two_plus_kids'] * df['post93']

# 3. Corremos la regresión incluyendo los controles de la pregunta 6
# Nota: Usamos 'one_kid' y 'two_plus_kids' en lugar de 'anykids' para separar los niveles base también.

model_hetero = smf.ols(
    formula='earn ~ one_kid + two_plus_kids + interact_1 + interact_2 + urate + nonwhite + age + ed + unearn + C(state) + C(year)',
    data=df
).fit()

print("--- RESULTADOS 7 (Efectos por N° de Hijos) ---")
# Mostramos las variables de interés
results_as_html_h = model_hetero.summary().tables[1].as_html()
result_df_h = pd.read_html(results_as_html_h, header=0, index_col=0)[0]
display(result_df_h.loc[['interact_1', 'interact_2']])

--- RESULTADOS 7 (Efectos por N° de Hijos) ---


,coef,std err,t,P>|t|,[0.025,0.975]
interact_1,2699.6643,795.516,3.394,0.001,1140.344,4258.984
interact_2,953.5636,694.884,1.372,0.170,-408.505,2315.632


- Contrario a la predicción basada en el monto del subsidio (que es mayor para familias grandes), los resultados muestran que la política fue efectiva solo para madres con un hijo (por el coeficiente de 2699.6, que tiene un valor p (0.001) significativo).
  
- En el mundo real, esto puede deberse a que los costos fijos y variables de trabajar (especialmente cuidado infantil y logística) crecen exponencialmente con el número de hijos. Es posible que el subsidio adicional por el segundo hijo no sea suficiente para compensar el costo de mercado de su cuidado, haciendo que la barrera para entrar al mercado laboral siga siendo demasiado alta para madres con familias numerosas.

## 8: Prueba Placebo (Datos pre-reforma)

In [ ]:
# 1. Filtramos datos: Usar solo años ANTERIORES a la reforma real (<= 1993)
df_placebo = df[df['year'] <= 1993].copy()

# 2. Creamos una reforma "falsa" en 1992
# Asumimos que la política empezó en 1992 (tratamiento: 92 y 93)
df_placebo['post92_fake'] = (df_placebo['year'] >= 1992).astype(int)

# 3. Creamos interacción placebo
df_placebo['interaction_placebo'] = df_placebo['anykids'] * df_placebo['post92_fake']

# 4. Corremos la regresión con controles
model_placebo = smf.ols(
    formula='earn ~ anykids + interaction_placebo + urate + nonwhite + age + ed + unearn + C(state) + C(year)',
    data=df_placebo
).fit()

print("\n--- RESULTADOS 8 (Prueba Placebo) ---")
results_as_html_p = model_placebo.summary().tables[1].as_html()
result_df_p = pd.read_html(results_as_html_p, header=0, index_col=0)[0]
display(result_df_p.loc[['interaction_placebo']])


--- RESULTADOS 8 (Prueba Placebo) ---


,coef,std err,t,P>|t|,[0.025,0.975]
interaction_placebo,942.4213,769.433,1.225,0.221,-565.888,2450.73


- Observamos que el coeficiente de interaction_placebo NO es significativo (por su valor-p de 0.2, que es mayor que 0.05).
- Ya que el coeficiente no es significativo, los resultados son tranquilizadores, ya que esto significa que antes de la reforma real, no había tendencias extrañas haciendo que las mujeres con hijos ganaran más.
- Esto valida que el efecto que encontramos en las preguntas anteriores fue causado realmente por la reforma de 1994 y no por otra cosa.